In [1]:
import pandas as pd
books=pd.read_csv('books.csv')

Decide how to handle empty values in the following columns:
ISBN
ISBN13
Language

In [2]:
books_less_missing=books.dropna(subset=['isbn','isbn13','language_code'])
book_index=[]
for i in range(len(books_less_missing)):
    book_index.append(i)
books_less_missing['new_id']=book_index
books_less_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8238 entries, 0 to 9998
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         8238 non-null   int64  
 1   book_id                    8238 non-null   int64  
 2   best_book_id               8238 non-null   int64  
 3   work_id                    8238 non-null   int64  
 4   books_count                8238 non-null   int64  
 5   isbn                       8238 non-null   object 
 6   isbn13                     8238 non-null   float64
 7   authors                    8238 non-null   object 
 8   original_publication_year  8225 non-null   float64
 9   original_title             7865 non-null   object 
 10  title                      8238 non-null   object 
 11  language_code              8238 non-null   object 
 12  average_rating             8238 non-null   float64
 13  ratings_count              8238 non-null   int64

<ipython-input-2-6c53eaf6d0b4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_less_missing['new_id']=book_index


Format ISBN 13 to match the specification with dashes. Consult the ISBN information page for guidance.

In [3]:
as_string_list=list(books_less_missing['isbn13'].astype(str))
# print((as_string))

as_isbn13_list=[]
for i in range(len(as_string_list)):
    decimal_location=as_string_list[i].find('.')
    string_no_decimal=as_string_list[i][0:decimal_location]
    isbn13_no_dashes="{0:0>13}".format(string_no_decimal)
    as_isbn13_list.append(f'{isbn13_no_dashes[0:3]}-{isbn13_no_dashes[3:4]}-{isbn13_no_dashes[4:9]}-{isbn13_no_dashes[9:12]}-{isbn13_no_dashes[12:13]}')
books_less_missing['isbn13']=as_isbn13_list


<ipython-input-3-f6ef5c1a50a8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_less_missing['isbn13']=as_isbn13_list


Add a column to show where original_title and title differ.

In [4]:
titles=list(books_less_missing['title'])
original_titles=list(books_less_missing['original_title'])
title_change_list=[]
for i in range(len(titles)):
    same_title=False
    if titles[i]==original_titles[i]:
        same_title=True
    title_change_list.append(same_title)
books_less_missing['title_changed']=title_change_list
books_less_missing.title_changed.value_counts()

<ipython-input-4-162be805341b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_less_missing['title_changed']=title_change_list


False    4981
True     3257
Name: title_changed, dtype: int64

Break up the authors into separate columns.

In [5]:
author_expansion=books_less_missing.authors.str.split(",",expand=True)
authors_index=[]
for i in range(len(author_expansion)):
    authors_index.append(i)
author_expansion["new_id"]=authors_index
expanded_authors=books_less_missing.merge(author_expansion,how='inner',on='new_id')


Store each publication by its publication year by century in its own worksheet. So 1800-1899, 1900-1999, 2000-2099, etc.

In [6]:
pre_17th_century=expanded_authors[expanded_authors.original_publication_year<1600]
post_16th_century=expanded_authors[expanded_authors.original_publication_year>=1600]
century17=post_16th_century[post_16th_century.original_publication_year<1700]
post_17th_century=post_16th_century[post_16th_century.original_publication_year>=1700]
century18=post_17th_century[post_17th_century.original_publication_year<1800]
post_18th_century=post_17th_century[post_17th_century.original_publication_year>=1800]
century19=post_17th_century[post_17th_century.original_publication_year<1900]
post_19th_century=post_17th_century[post_17th_century.original_publication_year>=1900]
century20=post_17th_century[post_17th_century.original_publication_year<2000]
post_20th_century=post_17th_century[post_17th_century.original_publication_year>=2000]

with pd.ExcelWriter('books_by_century.xlsx') as xlsxWriter:
    pre_17th_century.to_excel(xlsxWriter,sheet_name="pre 17th century")
    century17.to_excel(xlsxWriter,sheet_name="17th century")
    century18.to_excel(xlsxWriter,sheet_name="18th century")
    century19.to_excel(xlsxWriter,sheet_name="19th century")
    century20.to_excel(xlsxWriter,sheet_name="20th century")
    post_20th_century.to_excel(xlsxWriter,sheet_name="post 20th century")
    
xlsxWriter.save()

C:\Users\willi\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
